In [11]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import model
import datasets

from PIL import Image
import numpy as np

In [12]:
# Make sure to use the GPU. The following line is just a check to see if GPU is availables
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [13]:
# load your dataset and dataloader
# feel free to change header of bird_dataset class
root = 'path to images/'
root = 'birds_dataset/'
train_dataset = datasets.bird_dataset(root,'train_list.txt')
test_dataset = datasets.bird_dataset(root,'test_list.txt')

# Fill in optional arguments to the dataloader as you need it
train_dataloader = DataLoader(train_dataset)
test_dataloader = DataLoader(test_dataset)

044.Frigatebird/Frigatebird_0057_43016.jpg
044.Frigatebird/Frigatebird_0089_43013.jpg


In [16]:
# image = Image.open(root + '/images/' + '044.Frigatebird/Frigatebird_0057_43016.jpg')
# print(np.array(image).shape)

(333, 500, 3)


In [11]:
print(train_dataset.printstuff())

In [8]:
# Create NN model object
nn_model = model.baseline_Net(classes = 20)
nn_model.to(device)


baseline_Net(
  (b1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (b2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (b3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (b4): Sequential(
    (0): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
  )
  (avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Sequential(
    (0): Lin

In [9]:
# Create loss functions, optimizers
# For baseline model use this
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=1e-4)
# Initialize weights


In [10]:
# train your model
# For each epoch iterate over your dataloaders/datasets, pass it to your NN model, get output, calculate loss and
# backpropagate using optimizer


In [ ]:
# Save your model/best model